In [1]:
import multiprocessing
import math
from wiki_dump_reader import Cleaner , iterate
from tqdm import tqdm
import numpy as np
import multiprocessing
import nltk
from collections import Counter, defaultdict
from nltk import ngrams, word_tokenize
nltk.download('punkt')



[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
def minimize_pre(so,de):
    i=1000
    with open(so, 'r',encoding="utf-8") as sor:
        with open(de, 'w',encoding="utf-8") as des:
                for line in sor:
                    des.write(line[:(len(line))//i])

minimize_pre("PreProcessed_Corpus - Copy.txt", "PreProcessed_Corpus.txt")

Decorator to calculate execution time of functions.


In [3]:
import time

def calculate_time(func):
    """
    Decorator function to calculate the execution time of another function.

    Args:
        func (function): The function whose execution time is to be measured.

    Returns:
        function: A wrapped function that calculates and prints the execution time of the original function.

    """
    def wrapper(*args, **kwargs):
        """
        Calculate the execution time of the decorated function.

        Args:
            *args: Positional arguments to be passed to the decorated function.
            **kwargs: Keyword arguments to be passed to the decorated function.

        Returns:
            Any: The result of the decorated function.
        
        """
        # Record the start time before executing the function
        start_time = time.time()
        
        # Execute the decorated function
        result = func(*args, **kwargs)
        
        # Record the end time after executing the function
        end_time = time.time()
        
        # Calculate and print the execution time
        print(f"Execution time of {func.__name__}: {end_time - start_time} seconds")
        
        # Return the result of the decorated function
        return result
    return wrapper



Decorator to print the memory location of a wrapped function.


In [4]:
def memory_location(func):
    """
    A decorator function to print the memory location of a wrapped function.

    Args:
        func (function): The function whose memory location is to be printed.

    Returns:
        function: A wrapped function that prints the memory location of the original function and then calls it.

    """
    def wrapper(*args, **kwargs):
        """
        Print the memory location of the decorated function and then call it.

        Args:
            *args: Positional arguments to be passed to the decorated function.
            **kwargs: Keyword arguments to be passed to the decorated function.

        Returns:
            Any: The result of the decorated function.
        
        """
        print(f"Memory location of {func.__name__}: {id(func)}")
        return func(*args, **kwargs)
    return wrapper


Decorator to calculate execution time and memory location of the create_corpus function.


In [5]:
@calculate_time
@memory_location
def create_corpus():
    """
    Create a corpus from a Wikipedia dump file in Hindi.

    This function iterates over pages in a file named 'hiwiki-latest-pages-articles.xml',
    cleans the text, and writes it along with the title to a new file named 'Hindi_Corpus.txt'.
    It also updates a progress bar using tqdm to show the processing progress.

    Args:
        None

    Returns:
        None

    Raises:
        None

    """
    corpus_file = 'Hindi_Corpus.txt'  # File to store the corpus
    corpus_limit = 232729  # Limit for the number of pages in the corpus
    page_count = 0  # Counter for the number of processed pages
    cleaner = Cleaner()  # Assuming Cleaner class is imported or defined elsewhere.

    # Open the output file for writing
    with open(corpus_file, 'w', encoding='utf-8') as output:
        pg_bar = tqdm(total=corpus_limit)  # Initialize the progress bar
        # Iterate over pages in 'hiwiki-latest-pages-articles.xml'
        for title, text in iterate('hiwiki-latest-pages-articles.xml'):  # Assuming iterate function is defined elsewhere.
            # Clean the text
            text = cleaner.clean_text(text)
            cleaned_text, _ = cleaner.build_links(text)
            # Write title and cleaned text to the corpus file
            output.write(title + '\n' + cleaned_text + '\n')
            page_count += 1
            if page_count % 1000 == 0:
                pg_bar.update(1000)  # Update progress bar every 1000 pages
        pg_bar.close()  # Close the progress bar
        output.close()  # Close the output file
    print(f"\nPage count = {page_count}")  # Print total page count after processing
    
    create_corpus()



Preprocess a source file by removing stop words and non-Hindi characters.


In [6]:
def remove_stop_words(string):
    """
    Remove stop words from a given string.

    Args:
        string (str): Input string from which stop words are to be removed.

    Returns:
        str: String with stop words removed.
    """
    # Load stop words from file
    with open('stopwords.txt', 'r', encoding='utf-8') as stop:
        stop_words = stop.read().split()

    # Split the input string into words
    words = string.split()

    # Filter out stop words
    filtered_words = [word for word in words if word not in stop_words]

    # Join the filtered words back into a string
    return ' '.join(filtered_words)


import re

def remove_foreign(x):
    """
    Remove non-Hindi characters from a given string.

    Args:
        x (str): Input string from which non-Hindi characters are to be removed.

    Returns:
        str: String containing only Hindi characters.
    """
    # Split the input string into words
    words = x.split(' ')

    # Find Hindi characters using regex
    hindi_chars = [(re.compile(r'[\u0901-\u0939\u093C-\u094D\u0950-\u0954\u0958-\u0963\u097B-\u097F]')).findall(s) for s in words]

    # Join the Hindi characters back into a string
    hindi_string = ' '.join([''.join(chars) for chars in hindi_chars])

    return hindi_string


@calculate_time
@memory_location
def pre(source, destination):
    """
    Preprocess a source file and write the preprocessed content to a destination file.

    Args:
        source (str): Path to the source file.
        destination (str): Path to the destination file where preprocessed content will be written.

    Returns:
        None
    """
    line_count = 0
    # Open source file for reading and destination file for writing
    with open(source, 'r', encoding='utf-8') as input_file:
        with open(destination, 'w', encoding='utf-8') as output_file:
            # Initialize progress bar
            bar = tqdm(total=4400000)
            # Iterate over lines in the source file
            for line in input_file:
                # Remove newline characters
                string = line.strip('\n')
                # Remove non-Hindi characters
                string = remove_foreign(string)
                # Remove stop words
                string = remove_stop_words(string)
                # Write preprocessed string to destination file
                output_file.write(string)
                # Increment line count
                line_count += 1
                # Update progress bar every 10000 lines
                if line_count % 10000 == 0:
                    bar.update(10000)
            # Close progress bar
            bar.close()



In [ ]:
pre('Hindi_Corpus.txt', 'PreProcessed_Corpus.txt')

Generate vocabulary and distinct vocabulary lists from a preprocessed corpus file.


In [7]:
from collections import Counter

const_vocab = 20  # Global constant for minimum word frequency

def gen_vocab():
    """
    Generate a vocabulary list from the preprocessed corpus file.

    Reads the text from 'PreProcessed_Corpus.txt' and returns it as a list of words.

    Returns:
        list: A list of words extracted from the file 'PreProcessed_Corpus.txt'.
    
    """
    # Read text from the preprocessed corpus file
    with open('PreProcessed_Corpus.txt', 'r', encoding='utf-8') as r:
        text = r.read().split()
    
    # Count occurrences of each word
    word_counts = Counter(text)
    
    # Filter out words occurring less than const_vocab times
    text = [word for word in tqdm(text) if word_counts[word] > const_vocab]  
    
    return text

def gen_distinct_vocab():
    """
    Generate a list of distinct vocabulary from the preprocessed corpus file.

    Reads the text from 'PreProcessed_Corpus.txt', extracts distinct words, and returns them as a list.
    This function computes the vocabulary size and stores it in a global variable 'vocabulary_size'.

    Returns:
        list: A list of distinct words extracted from the file 'PreProcessed_Corpus.txt'.
    
    """
    # Read text from the preprocessed corpus file
    with open('PreProcessed_Corpus.txt', 'r', encoding='utf-8') as r:
        text = r.read().split()
        
        # Count occurrences of each word
        counter = Counter(text)
        
        # Filter out words occurring less than const_vocab times
        reduced_vocab = [item for item, count in counter.items() if count > const_vocab]
        
        # Convert to set to get distinct words
        text_set = set(reduced_vocab)
        
        # Calculate vocabulary size and store it in a global variable
        global vocabulary_size
        vocabulary_size = len(text_set)
        
    return list(text_set)


In [8]:

global words_list
words_list=gen_vocab()
distinct_vocab=gen_distinct_vocab()
vocabulary_size = len(distinct_vocab)
print((vocabulary_size))

100%|██████████| 42667/42667 [00:00<00:00, 3023047.55it/s]

316


In [9]:

def gen_grams():            
    global tokens
    global grams
    tokens = word_tokenize(' '.join(words_list))
    grams = ngrams(tokens, 6)
    return None 

gen_grams()

In [10]:
print("Number of tokens is: ", len(tokens))
print("Size of the vocabulary: ", vocabulary_size)
print("First 1500 words of the Vocabulary: ", words_list[:1500])


Number of tokens is:  14911
Size of the vocabulary:  316
First 1500 words of the Vocabulary:  ['मुख्य', 'हिंदी', 'गीत', 'गीत', 'सदी', 'स्वर', 'बना', 'गीत', 'माना', 'पूजा', 'विधि', 'पूजा', 'हिन्दू', 'धर्म', 'विभिन्न', 'देवताओं', 'पूजा', 'तीन', 'प्रकार', 'नाम', 'वैदिक', 'देवता', 'नाम', 'नमः', 'वेदों', 'वाली', 'वाली', 'वाली', 'वाली', 'वाली', 'वाली', 'देवता', 'पूजा', 'नाम', 'संस्कृत', 'पूजा', 'मुख्य', 'देवताओं', 'पूजा', 'नाम', 'ॐ', 'सिद्धि', 'विनायकाय', 'ॐ', 'ॐ', 'ॐ', 'ॐ', 'ॐ', 'नमः', 'ॐ', 'ॐ', 'नमः', 'ॐ', 'ॐ', 'ॐ', 'ॐ', 'नमः', 'ॐ', 'नमः', 'ॐ', 'ॐ', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ', 'सिद्धि', 'विनायकाय', 'नमः', 'समर्पयामि', 'ॐ',

Create co-occurrence matrix based on grams with timing and memory measurement.


In [11]:

cocrmat = np.zeros((len(distinct_vocab),len(distinct_vocab)),dtype=np.int64)

@calculate_time
@memory_location
def create_matrix():
	
	ramp=[0,1,2,3,4,5]

	for gram in tqdm(grams):

		x=list(gram)

		if x[0] in distinct_vocab:
			for i in range(1,len(x)):
				if x[i] in distinct_vocab:
					cocrmat[distinct_vocab.index(x[0])][distinct_vocab.index(x[i])] += 5-ramp[i]
					
		x.reverse()
	
		if x[0] in distinct_vocab:
			for i in range(1,len(x)):
				if x[i] in distinct_vocab:
					cocrmat[distinct_vocab.index(x[0])][distinct_vocab.index(x[i])] += 5-ramp[i]
		
create_matrix()


Memory location of create_matrix: 1398453161632


14906it [00:00, 19769.20it/s]

Execution time of wrapper: 0.7540011405944824 seconds


Calculate probability and generate PPMI matrix with timing and memory measurement.


In [12]:

@calculate_time
@memory_location
def calculate_probability(matrix):
    
    # Calculate row sums and column sums
    row_sums = np.sum(matrix, axis=1)
    col_sums = np.sum(matrix, axis=0)

    # Total co-occurrences
    N = np.sum(matrix)

    if(N==0):
        return 0,0

    # Calculate pi and pj
    pI = row_sums / N
    pJ = col_sums / N

    global pi,pj

    pi,pj=pI,pJ
 
 
@calculate_time
@memory_location
def gen_ppmi_matrix(matrix):
    co_occurrence_matrix=matrix
 
    ppmi = np.zeros((len(distinct_vocab), len(distinct_vocab)))

    row_sums = np.sum(co_occurrence_matrix, axis=1)
    for i in tqdm(range(len(distinct_vocab))):
        for j in range(len(distinct_vocab)):

            if (row_sums[i] == 0 or co_occurrence_matrix[i][j] == 0):
                ppmi[i][j] = 0
                continue
            pij = ((co_occurrence_matrix[i][j]))/row_sums[i]
            ppmi[i][j] = max(0, math.log2(pij/(pi[i]*pj[j])))

    return ppmi


Calculate probability and generate PPMI matrix.


In [13]:
calculate_probability(cocrmat)
ppmi_matrix=gen_ppmi_matrix(cocrmat)


Memory location of calculate_probability: 1398453165952
Execution time of wrapper: 0.002025604248046875 seconds
Memory location of gen_ppmi_matrix: 1398453163872


100%|██████████| 316/316 [00:00<00:00, 3635.70it/s]

Execution time of wrapper: 0.09158563613891602 seconds


Script to find the most common elements in a list.


In [14]:
from collections import Counter

my_list = gen_vocab()
counter = Counter(my_list)
common_elements = counter.most_common(10)
most_common_elements=[a for (a,_) in common_elements]

  0%|          | 0/42667 [00:00<?, ?it/s]

100%|██████████| 42667/42667 [00:00<00:00, 2490721.90it/s]


Script to find nearest neighbors for each noun in a given list.


In [15]:
import math

def insert_into_sorted_list(sorted_list, element):
	index = 0
	while index < len(sorted_list) and sorted_list[index][1] > element[1]:
		index += 1
	sorted_list.insert(index, element)
	return sorted_list[:-1]

def cosine(a,b):
		mag_a = math.sqrt(sum(component ** 2 for component in a))
		mag_b = math.sqrt(sum(component ** 2 for component in b))
		dot_product = sum(ai * bi for ai, bi in zip(a, b))
		if mag_a == 0 or mag_b ==0:
			return 0
		else:
			return dot_product/(mag_a*mag_b)

def find_nearest_neighbor_of_noun(index):
	l=[(distinct_vocab[0],(cosine(ppmi_matrix[index], ppmi_matrix[0]))) for _ in range(11)]
	for i in tqdm(range(1,len(distinct_vocab))):
		if i!=index and cosine(ppmi_matrix[index],ppmi_matrix[i])>l[10][1]:
				l=insert_into_sorted_list(l,(distinct_vocab[i],(cosine(ppmi_matrix[index], ppmi_matrix[i]))))
	return l

nearest_neighbour_dict={}

@calculate_time
@memory_location
def nearest_seq():
	for x in tqdm(most_common_elements):
		nearest_neighbour_dict[x]=find_nearest_neighbor_of_noun(distinct_vocab.index(x))

nearest_seq()

for i in nearest_neighbour_dict:
	print("Words similar to ",i," :")
	for j in nearest_neighbour_dict[i]:
		print ("  ",j[0]," : ",j[1])


Memory location of nearest_seq: 1398453166752


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 13.39it/s]

Execution time of wrapper: 0.7549777030944824 seconds
Words similar to  रूप  :
   नहीं  :  0.9419451759461548
   भारतीय  :  0.889302376371782
   भारत  :  0.8725292899568924
   नाम  :  0.8658936670580452
   अनुसार  :  0.8584788816049529
   सबसे  :  0.857209523336516
   कारण  :  0.8563443062949164
   शब्द  :  0.8556091594530136
   प्रकार  :  0.8464437759920724
   भाषा  :  0.8440236096542005
   अर्थ  :  0.8281050715542241
Words similar to  भारत  :
   रूप  :  0.8725292899568924
   भारतीय  :  0.871773948585898
   नहीं  :  0.8518336004742406
   सबसे  :  0.8403134833450826
   भाषा  :  0.838068788753336
   क्षेत्र  :  0.8274030404852076
   अनुसार  :  0.7955146150217367
   धर्म  :  0.782202244633452
   नाम  :  0.782119156730583
   संस्कृत  :  0.7808500169542222
   हिन्दी  :  0.7806108756572399
Words similar to  नहीं  :
   रूप  :  0.9419451759461548
   भारतीय  :  0.8607626511172397
   सबसे  :  0.8599672371073805
   नाम  :  0.8576865717395105
   शब्द  :  0.8546129127629498
   भारत  :  0.851833600

Parallelized co-occurrence matrix generation using multiprocessing.


In [16]:
n = vocabulary_size  # Size of the vocabulary
p_mat = multiprocessing.Array('d', n*n)  # Shared memory array to store partial co-occurrence matrix

def compile_grams(gram, p_mat):
    """
    Compile co-occurrence counts for a gram and update the shared memory array.

    Args:
        gram (tuple): The gram to compile co-occurrence counts for.
        p_mat (multiprocessing.Array): Shared memory array to store partial co-occurrence matrix.

    Returns:
        None
    """
    ramp = [0, 1, 2, 3, 4, 5]  # Ramp sequence for weight calculation
    leng = len(distinct_vocab)  # Length of the distinct vocabulary

    x = list(gram)

    # Forward pass
    if x[0] in distinct_vocab:
        for i in range(1, len(x)):
            if x[i] in distinct_vocab:
                # Update co-occurrence count based on weight
                p_mat[distinct_vocab.index(x[0]) * leng + distinct_vocab.index(x[i])] += 5 - ramp[i]

    x.reverse()

    # Backward pass
    if x[0] in distinct_vocab:
        for i in range(1, len(x)):
            if x[i] in distinct_vocab:
                # Update co-occurrence count based on weight
                p_mat[distinct_vocab.index(x[0]) * leng + distinct_vocab.index(x[i])] += 5 - ramp[i]

@calculate_time
@memory_location
def create_matrix_p(p_mat):
    """
    Create a co-occurrence matrix in parallel using multiprocessing.

    Args:
        p_mat (multiprocessing.Array): Shared memory array to store partial co-occurrence matrix.

    Returns:
        None
    """
    grams = ngrams(tokens, 6)  # Generate ngrams of size 6
    jobs = []

    # Create a process for each gram to compile co-occurrence counts
    for gram in tqdm(grams):
        p = multiprocessing.Process(target=compile_grams, args=(gram, p_mat,))
        p.start()
        jobs.append(p)

    # Wait for all processes to finish
    for job in jobs:
        job.join()

# Create the co-occurrence matrix in parallel
create_matrix_p(p_mat)

# Convert the shared memory array to a numpy array
p_matrix = np.zeros((n, n), dtype=np.int64)
for i in range(n):
    for j in range(n):
        p_matrix[i][j] = p_mat[i * n + j]


Memory location of create_matrix_p: 1399103470016


0it [00:00, ?it/s]

14906it [03:07, 79.36it/s] 


Execution time of wrapper: 188.86974334716797 seconds



Parallel Nearest Neighbor Search

In [17]:
import math
import multiprocessing

# Using a multiprocessing Manager to create a shared dictionary
neigh = multiprocessing.Manager().dict()

def p_insert_into_sorted_list(sorted_list, element):
    """
    Insert an element into a sorted list in descending order based on the second value of each tuple.

    Args:
        sorted_list (list): The sorted list to insert the element into.
        element (tuple): The element to insert into the sorted list.

    Returns:
        list: The sorted list with the new element inserted.
    """
    index = 0
    while index < len(sorted_list) and sorted_list[index][1] > element[1]:
        index += 1
    sorted_list.insert(index, element)
    return sorted_list[:-1]

def p_cosine(a, b):
    """
    Calculate the cosine similarity between two vectors.

    Args:
        a (list): First vector.
        b (list): Second vector.

    Returns:
        float: Cosine similarity between the two vectors.
    """
    mag_a = math.sqrt(sum(component ** 2 for component in a))
    mag_b = math.sqrt(sum(component ** 2 for component in b))
    dot_product = sum(ai * bi for ai, bi in zip(a, b))
    if mag_a == 0 or mag_b == 0:
        return 0
    else:
        return dot_product / (mag_a * mag_b)

def p_find_nearest_neighbor_of_noun(index, neigh):
    """
    Find the nearest neighbors of a noun in the ppmi_matrix in parallel.

    Args:
        index (int): Index of the noun in the distinct_vocab.
        neigh (multiprocessing.Manager().dict): Shared dictionary to store nearest neighbors.

    Returns:
        None
    """
    # Initialize a list with the first noun and its cosine similarity score
    l = [(distinct_vocab[0], (p_cosine(ppmi_matrix[index], ppmi_matrix[0]))) for _ in range(11)]
    
    # Iterate over other nouns
    for i in tqdm(range(1, n)):
        if i != index:
            if p_cosine(ppmi_matrix[index], ppmi_matrix[i]) > l[10][1]:
                # Update the list with the nearest neighbors
                l = p_insert_into_sorted_list(l, (distinct_vocab[i], (p_cosine(ppmi_matrix[index], ppmi_matrix[i]))))

    # Store the list of nearest neighbors in the shared dictionary
    neigh[distinct_vocab[index]] = l

@calculate_time
@memory_location
def parallel_nearest():
    """
    Find nearest neighbors for each noun in most_common_elements in parallel.

    Returns:
        None
    """
    jobs = []
    for x in (most_common_elements):
        # Create a process for each noun to find nearest neighbors
        p = multiprocessing.Process(target=p_find_nearest_neighbor_of_noun, args=(distinct_vocab.index(x), neigh,))
        p.start()
        jobs.append(p)

    # Wait for all processes to finish
    for job in jobs:
        job.join()

# Execute parallel_nearest function to find nearest neighbors for each noun in parallel
parallel_nearest()

# Print the shared dictionary containing nearest neighbors for each noun
print(neigh)


Memory location of nearest_seq: 1398453166752


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 13.39it/s]

Execution time of wrapper: 0.0562656752178425 seconds
Words similar to  रूप  :
   नहीं  :  0.9419451759461548
   भारतीय  :  0.889302376371782
   भारत  :  0.8725292899568924
   नाम  :  0.8658936670580452
   अनुसार  :  0.8584788816049529
   सबसे  :  0.857209523336516
   कारण  :  0.8563443062949164
   शब्द  :  0.8556091594530136
   प्रकार  :  0.8464437759920724
   भाषा  :  0.8440236096542005
   अर्थ  :  0.8281050715542241
Words similar to  भारत  :
   रूप  :  0.8725292899568924
   भारतीय  :  0.871773948585898
   नहीं  :  0.8518336004742406
   सबसे  :  0.8403134833450826
   भाषा  :  0.838068788753336
   क्षेत्र  :  0.8274030404852076
   अनुसार  :  0.7955146150217367
   धर्म  :  0.782202244633452
   नाम  :  0.782119156730583
   संस्कृत  :  0.7808500169542222
   हिन्दी  :  0.7806108756572399
Words similar to  नहीं  :
   रूप  :  0.9419451759461548
   भारतीय  :  0.8607626511172397
   सबसे  :  0.8599672371073805
   नाम  :  0.8576865717395105
   शब्द  :  0.8546129127629498
   भारत  :  0.851833600